In [ ]:
%run 

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import warnings
import gc
# enable garbage collector to aid in memory 
gc.enable()
# eliminate future warnings
warnings.filterwarnings('ignore')


# Import Data

In [2]:
aisles = pd.read_csv("aisles.csv")
department = pd.read_csv("departments.csv")
orders = pd.read_csv("orders.csv")
prior = pd.read_csv("order_products__prior.csv")
train = pd.read_csv("order_products__train.csv")
products = pd.read_csv("products.csv")

In [3]:
# stock data frame for product and user_product features used later
stock = products.merge(department, on = 'department_id', how = 'left')
stock = products.merge(aisles, on = 'aisle_id', how = 'left')
stock.drop(['aisle', 'product_name'], axis = 1, inplace = True)
stock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49688 entries, 0 to 49687
Data columns (total 3 columns):
product_id       49688 non-null int64
aisle_id         49688 non-null int64
department_id    49688 non-null int64
dtypes: int64(3)
memory usage: 1.5 MB


In [4]:
# prior_orders for user features, joins user_id and order_id 
prior_orders = prior.merge(orders, on = 'order_id', how = 'inner')

In [5]:
del products, aisles, department
gc.collect()

40

In [6]:
def memory_cruncher(df):
    """Reduces memory usage of data frame by converting the columns in to smaller dtypes"""
    for c in df.columns:
        if df[c].dtype == 'float64':
            df[c] = df[c].astype('float32')
        elif df[c].dtype == 'int64':
            df[c] = df[c].astype('int32')
        elif c == 'reordered':
            df[c] == df[c].astype('int8')
        else:
            pass
    return df

In [7]:
memory_cruncher(prior_orders);
memory_cruncher(stock);

In [8]:
prior_orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32434489 entries, 0 to 32434488
Data columns (total 10 columns):
order_id                  int32
product_id                int32
add_to_cart_order         int32
reordered                 int32
user_id                   int32
eval_set                  object
order_number              int32
order_dow                 int32
order_hour_of_day         int32
days_since_prior_order    float32
dtypes: float32(1), int32(8), object(1)
memory usage: 1.6+ GB


In [9]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49688 entries, 0 to 49687
Data columns (total 3 columns):
product_id       49688 non-null int32
aisle_id         49688 non-null int32
department_id    49688 non-null int32
dtypes: int32(3)
memory usage: 970.5 KB


# Predictor Variables of Market Basket Analysis
<ol>
  1. User prediction variables - behaviors of users <br>
  2. Product prediction variables - info on products <br>
  3. User/Product prediction variables - behavior towards a product
</ol>

# User Predictor Variables

In [10]:
# total amount of orders by user 
user = prior_orders.groupby('user_id').order_number.max().to_frame('user_totals_u').reset_index()

In [11]:
# average number of orders per user
number_of_orders = prior_orders.groupby('user_id').order_number.mean().to_frame('number_of_orders_u').reset_index()

In [12]:
#avg number of days between orders
avg_days_between = prior_orders.groupby('user_id').days_since_prior_order.mean().to_frame('average_days_between_u').reset_index()

In [13]:
# average number for reordered products by user
reorder_ratio = prior_orders.groupby('user_id').reordered.mean().to_frame('reorder_ratio_u').reset_index()

In [14]:
# order day of the week with the most items 
most_dow = prior_orders.groupby('user_id').order_dow.agg(lambda x:x.value_counts().index[0]).to_frame('most_dow_u').reset_index()

In [15]:
# order hour of the day with the most items
most_hour = prior_orders.groupby('user_id').order_hour_of_day.agg(lambda x:x.value_counts().index[0]).to_frame('most_hour_u').reset_index()

In [16]:
# total amount of different items bought
specific_items = prior_orders.groupby('user_id').size().to_frame('specific_items_p').reset_index()

In [17]:
user = user.merge(number_of_orders, on = 'user_id', how = 'left')
user = user.merge(avg_days_between, on = 'user_id', how = 'left')
user = user.merge(most_dow, on = 'user_id', how = 'left')
user = user.merge(most_hour, on = 'user_id', how = 'left')
user = user.merge(reorder_ratio, on = 'user_id', how = 'left')
user = user.merge(specific_items, on = 'user_id', how = 'left')

In [18]:
memory_cruncher(user)
user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206209 entries, 0 to 206208
Data columns (total 8 columns):
user_id                   206209 non-null int32
user_totals_u             206209 non-null int32
number_of_orders_u        206209 non-null float32
average_days_between_u    206209 non-null float32
most_dow_u                206209 non-null int32
most_hour_u               206209 non-null int32
reorder_ratio_u           206209 non-null float32
specific_items_p          206209 non-null int32
dtypes: float32(3), int32(5)
memory usage: 7.9 MB


In [19]:
del number_of_orders, avg_days_between, most_dow, most_hour, reorder_ratio, specific_items
gc.collect()

40

# Product predictor variables 

In [20]:
# total amount of purchases per product
product = prior_orders.groupby('product_id').order_id.count().to_frame('total_purchased_p').reset_index()
product.head()

,product_id,total_purchased_p
0,1,1852
1,2,90
2,3,277
3,4,329
4,5,15


In [21]:
# calculate the ratio of repurchase for each product
reorder_ratio = prior_orders.groupby('product_id').reordered.mean().to_frame('reorder_ratio_p').reset_index()
reorder_ratio.head()


,product_id,reorder_ratio_p
0,1,0.613391
1,2,0.133333
2,3,0.732852
3,4,0.446809
4,5,0.600000


In [22]:
# calculate the mean for position that the product is added to the cart
addtocart = prior_orders.groupby('product_id').add_to_cart_order.mean().to_frame('cart_position_p').reset_index()
addtocart.head()


,product_id,cart_position_p
0,1,5.801836
1,2,9.888889
2,3,6.415162
3,4,9.507599
4,5,6.466667


In [23]:
#Merge the product with reorder probability
product = product.merge(reorder_ratio, on='product_id', how='left')
# merge product with addtocart
product = product.merge(addtocart, on = 'product_id', how = 'left')

product.head()

,product_id,total_purchased_p,reorder_ratio_p,cart_position_p
0,1,1852,0.613391,5.801836
1,2,90,0.133333,9.888889
2,3,277,0.732852,6.415162
3,4,329,0.446809,9.507599
4,5,15,0.600000,6.466667


In [24]:
memory_cruncher(product);

In [25]:
del reorder_ratio, addtocart
gc.collect()

40

# User-Product predictor variables

In [26]:
# amount of times a user bought a specific product
user_product = prior_orders.groupby(by=['user_id', 'product_id']).order_id.count().to_frame('times_bought_up').reset_index()
# changing the dtype.
user_product['times_bought_up'] = user_product['times_bought_up'].astype(np.uint16)
user_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13307953 entries, 0 to 13307952
Data columns (total 3 columns):
user_id            int64
product_id         int64
times_bought_up    uint16
dtypes: int64(2), uint16(1)
memory usage: 228.4 MB


In [27]:
# how many times a user purchased the product after purchasing it once, same as above used to calculate ratio, support
times = prior_orders.groupby(['user_id','product_id']).order_id.count().to_frame('amt_bought').reset_index()
times.head()

,user_id,product_id,amt_bought
0,1,196,10
1,1,10258,9
2,1,10326,1
3,1,12427,10
4,1,13032,3


In [28]:
# total orders used for calculating range, support
total_orders = prior_orders.groupby('user_id')['order_number'].max().to_frame('total_orders').reset_index()
total_orders.head()

,user_id,total_orders
0,1,10
1,2,14
2,3,12
3,4,5
4,5,4


In [29]:
# finding when the user has bought a product the first time.
first_order_num = prior_orders.groupby(by=['user_id', 'product_id'])['order_number'].min().to_frame('first_order_num').reset_index()
first_order_num.head()

,user_id,product_id,first_order_num
0,1,196,1
1,1,10258,2
2,1,10326,5
3,1,12427,1
4,1,13032,2


In [30]:
# merging to make calculations on 
span = pd.merge(total_orders, first_order_num, on='user_id', how='right')
span.head()

,user_id,total_orders,product_id,first_order_num
0,1,10,196,1
1,1,10,10258,2
2,1,10,10326,5
3,1,10,12427,1
4,1,10,13032,2


In [31]:
# calculating range, plus one for offset
span['range'] = span.total_orders - span.first_order_num + 1
span.head()

,user_id,total_orders,product_id,first_order_num,range
0,1,10,196,1,10
1,1,10,10258,2,9
2,1,10,10326,5,6
3,1,10,12427,1,10
4,1,10,13032,2,9


In [32]:
# merging times df with the span to calculate reorder ratio
up_ratio = pd.merge(times, span, on=['user_id', 'product_id'], how='left')
up_ratio.head()

,user_id,product_id,amt_bought,total_orders,first_order_num,range
0,1,196,10,10,1,10
1,1,10258,9,10,2,9
2,1,10326,1,10,5,6
3,1,12427,10,10,1,10
4,1,13032,3,10,2,9


In [33]:
#calculating the ratio.
up_ratio['reorder_ratio_up'] = up_ratio.amt_bought / up_ratio.range
up_ratio.head()

,user_id,product_id,amt_bought,total_orders,first_order_num,range,reorder_ratio_up
0,1,196,10,10,1,10,1.000000
1,1,10258,9,10,2,9,1.000000
2,1,10326,1,10,5,6,0.166667
3,1,12427,10,10,1,10,1.000000
4,1,13032,3,10,2,9,0.333333


In [34]:
# droppinf irrelevant columns
up_ratio.drop(['amt_bought', 'total_orders', 'first_order_num', 'range'], axis=1, inplace=True)
up_ratio.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 3 columns):
user_id             int64
product_id          int64
reorder_ratio_up    float64
dtypes: float64(1), int64(2)
memory usage: 406.1 MB


In [35]:
# deleting all the unwanted df.
del span, first_order_num, total_orders
gc.collect()

124

In [36]:
# merging up ratio to construct user-product df
user_product = user_product.merge(up_ratio, on=['user_id', 'product_id'], how='left')
user_product.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 4 columns):
user_id             int64
product_id          int64
times_bought_up     uint16
reorder_ratio_up    float64
dtypes: float64(1), int64(2), uint16(1)
memory usage: 431.5 MB


In [37]:
# Reversing the order number for each product.
prior_orders['reversed_order'] = prior_orders.groupby(by=['user_id'])['order_number'].transform(max) - prior_orders.order_number + 1

In [38]:
# keeping only the first 5 orders from the order_number_back.
last_5 = prior_orders.loc[prior_orders.reversed_order <= 5]
last_5.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,reversed_order
56,6,40462,1,0,22352,prior,4,1,12,30.0,5
57,6,15873,2,0,22352,prior,4,1,12,30.0,5
58,6,41897,3,0,22352,prior,4,1,12,30.0,5
59,7,34050,1,0,142903,prior,11,2,14,30.0,2
60,7,46802,2,0,142903,prior,11,2,14,30.0,2


In [39]:
# product bought by users in the last_five orders.
last_five = last_5.groupby(by=['user_id', 'product_id'])['order_id'].count().to_frame('last_five_up').reset_index()
last_five.head()

,user_id,product_id,last_five_up
0,1,196,5
1,1,10258,5
2,1,12427,5
3,1,13032,2
4,1,25133,5


In [40]:
# ratio of the products bought in the last_five orders.
last_five['ratio_last_five_up'] = last_five.last_five_up / 5.0
last_five.head()

,user_id,product_id,last_five_up,ratio_last_five_up
0,1,196,5,1.0
1,1,10258,5,1.0
2,1,12427,5,1.0
3,1,13032,2,0.4
4,1,25133,5,1.0


In [41]:
# merging this feature with uxp df.
user_product = user_product.merge(last_five, on=['user_id', 'product_id'], how='left')

In [42]:
del last_five, last_5
gc.collect()

53

In [43]:
# filling the NAN values with 0.
user_product.fillna(0, inplace=True)
memory_cruncher(user_product)
user_product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 6 columns):
user_id               int32
product_id            int32
times_bought_up       uint16
reorder_ratio_up      float32
last_five_up          float32
ratio_last_five_up    float32
dtypes: float32(3), int32(2), uint16(1)
memory usage: 380.7 MB


#  merge user, product, user-product df

In [44]:
# Merge user-product features with the user features
df1 = user_product.merge(user, on='user_id', how='left')
# Merging product features with df for a complete df with all three categories of predictor variables
df1 = df1.merge(product, on='product_id', how='left')
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 16 columns):
user_id                   int32
product_id                int32
times_bought_up           uint16
reorder_ratio_up          float32
last_five_up              float32
ratio_last_five_up        float32
user_totals_u             int32
number_of_orders_u        float32
average_days_between_u    float32
most_dow_u                int32
most_hour_u               int32
reorder_ratio_u           float32
specific_items_p          int32
total_purchased_p         int32
reorder_ratio_p           float32
cart_position_p           float32
dtypes: float32(8), int32(7), uint16(1)
memory usage: 888.4 MB


In [45]:
del user, product, user_product, prior
gc.collect()

60

# Preprocessing

In [46]:
# extracting training and test for identifying in df
future_orders = orders.loc[((orders.eval_set == 'train') | (orders.eval_set == 'test')), ['user_id', 'eval_set', 'order_id']]
future_orders.head()

,user_id,eval_set,order_id
10,1,train,1187899
25,2,train,1492625
38,3,test,2774568
44,4,test,329954
49,5,train,2196797


In [47]:
# test and train merged with df1 to allow for seperation
df1 = df1.merge(future_orders, on = 'user_id', how = 'left')
df1.head()

,user_id,product_id,times_bought_up,reorder_ratio_up,last_five_up,ratio_last_five_up,user_totals_u,number_of_orders_u,average_days_between_u,most_dow_u,most_hour_u,reorder_ratio_u,specific_items_p,total_purchased_p,reorder_ratio_p,cart_position_p,eval_set,order_id
0,1,196,10,1.000000,5.0,1.0,10,5.81356,20.25926,4,7,0.694915,59,35791,0.776480,3.721774,train,1187899
1,1,10258,9,1.000000,5.0,1.0,10,5.81356,20.25926,4,7,0.694915,59,1946,0.713772,4.277493,train,1187899
2,1,10326,1,0.166667,0.0,0.0,10,5.81356,20.25926,4,7,0.694915,59,5526,0.652009,4.191097,train,1187899
3,1,12427,10,1.000000,5.0,1.0,10,5.81356,20.25926,4,7,0.694915,59,6476,0.740735,4.760037,train,1187899
4,1,13032,3,0.333333,2.0,0.4,10,5.81356,20.25926,4,7,0.694915,59,3751,0.657158,5.622767,train,1187899


In [48]:
# seperation of training rows
df_train = df1[df1.eval_set == 'train']
df_train.head()

,user_id,product_id,times_bought_up,reorder_ratio_up,last_five_up,ratio_last_five_up,user_totals_u,number_of_orders_u,average_days_between_u,most_dow_u,most_hour_u,reorder_ratio_u,specific_items_p,total_purchased_p,reorder_ratio_p,cart_position_p,eval_set,order_id
0,1,196,10,1.000000,5.0,1.0,10,5.81356,20.25926,4,7,0.694915,59,35791,0.776480,3.721774,train,1187899
1,1,10258,9,1.000000,5.0,1.0,10,5.81356,20.25926,4,7,0.694915,59,1946,0.713772,4.277493,train,1187899
2,1,10326,1,0.166667,0.0,0.0,10,5.81356,20.25926,4,7,0.694915,59,5526,0.652009,4.191097,train,1187899
3,1,12427,10,1.000000,5.0,1.0,10,5.81356,20.25926,4,7,0.694915,59,6476,0.740735,4.760037,train,1187899
4,1,13032,3,0.333333,2.0,0.4,10,5.81356,20.25926,4,7,0.694915,59,3751,0.657158,5.622767,train,1187899


In [49]:
# retrieves the reorder column from the train df
df_train = df_train.merge(train[['product_id', 'order_id', 'reordered']], on = ['product_id', 'order_id'], how = 'left')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8474661 entries, 0 to 8474660
Data columns (total 19 columns):
user_id                   int32
product_id                int32
times_bought_up           uint16
reorder_ratio_up          float32
last_five_up              float32
ratio_last_five_up        float32
user_totals_u             int32
number_of_orders_u        float32
average_days_between_u    float32
most_dow_u                int32
most_hour_u               int32
reorder_ratio_u           float32
specific_items_p          int32
total_purchased_p         int32
reorder_ratio_p           float32
cart_position_p           float32
eval_set                  object
order_id                  int64
reordered                 float64
dtypes: float32(8), float64(1), int32(7), int64(1), object(1), uint16(1)
memory usage: 759.7+ MB


In [50]:
# order_id and eval set are no longer relevant
df_train.drop(['order_id', 'eval_set'], axis = 1, inplace = True)

In [51]:
# fill all NaN with zero, mostly reordered column
df_train.fillna(0, inplace = True)

In [52]:
# seperate test rows
df_test = df1[df1.eval_set == 'test']
df_test.head()

,user_id,product_id,times_bought_up,reorder_ratio_up,last_five_up,ratio_last_five_up,user_totals_u,number_of_orders_u,average_days_between_u,most_dow_u,most_hour_u,reorder_ratio_u,specific_items_p,total_purchased_p,reorder_ratio_p,cart_position_p,eval_set,order_id
120,3,248,1,0.090909,0.0,0.0,12,6.0,11.48718,0,16,0.625,88,6371,0.400251,10.620782,test,2774568
121,3,1005,1,0.333333,1.0,0.2,12,6.0,11.48718,0,16,0.625,88,463,0.440605,9.498920,test,2774568
122,3,1819,3,0.333333,0.0,0.0,12,6.0,11.48718,0,16,0.625,88,2424,0.492162,9.287541,test,2774568
123,3,7503,1,0.100000,0.0,0.0,12,6.0,11.48718,0,16,0.625,88,12474,0.553551,9.547379,test,2774568
124,3,8021,1,0.090909,0.0,0.0,12,6.0,11.48718,0,16,0.625,88,27864,0.591157,8.822854,test,2774568


In [53]:
# order+id and eval set no longer relevant to test df either
df_test.drop(['eval_set', 'order_id'], axis = 1, inplace = True)

/home/johnmketterer/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [54]:
# adding columns with aisle id and department id to be encoded 
df_train = df_train.merge(stock, on = 'product_id', how = 'left')
df_test = df_test.merge(stock, on = 'product_id', how = 'left')

In [55]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4833292 entries, 0 to 4833291
Data columns (total 18 columns):
user_id                   int32
product_id                int32
times_bought_up           uint16
reorder_ratio_up          float32
last_five_up              float32
ratio_last_five_up        float32
user_totals_u             int32
number_of_orders_u        float32
average_days_between_u    float32
most_dow_u                int32
most_hour_u               int32
reorder_ratio_u           float32
specific_items_p          int32
total_purchased_p         int32
reorder_ratio_p           float32
cart_position_p           float32
aisle_id                  int32
department_id             int32
dtypes: float32(8), int32(9), uint16(1)
memory usage: 359.5 MB


In [56]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8474661 entries, 0 to 8474660
Data columns (total 19 columns):
user_id                   int32
product_id                int32
times_bought_up           uint16
reorder_ratio_up          float32
last_five_up              float32
ratio_last_five_up        float32
user_totals_u             int32
number_of_orders_u        float32
average_days_between_u    float32
most_dow_u                int32
most_hour_u               int32
reorder_ratio_u           float32
specific_items_p          int32
total_purchased_p         int32
reorder_ratio_p           float32
cart_position_p           float32
reordered                 float64
aisle_id                  int32
department_id             int32
dtypes: float32(8), float64(1), int32(9), uint16(1)
memory usage: 695.1 MB


In [62]:
del future_orders, df1, 